In [1]:
import os
from datetime import datetime

from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib 

matplotlib.rcParams.update({'font.size': 15})
%matplotlib inline

import plotly
import plotly.graph_objs as pgo

from nxplot_3d import get_trace_nodes_3D_legend
from nxplot_3d import get_trace_edges_3D
from nxplot_3d import plot_3D

from export_import import export_data
from export_import import read_data

from csv_to_network import get_graph

In [2]:
path= '../data/dynamic/'
file_names =  sorted(os.listdir(path), key=lambda x: datetime.strptime(x, '%Y-%m.csv'))
dframes = {}

for fname in file_names:
   dframes[fname] = pd.read_csv(path+ fname, sep='|', keep_default_na=False)

dtgraphs = {}; i = 0
for fname in dframes:
    dtgraphs[fname[:-4]] = get_graph(dframes[fname], cut=0)
    dtgraphs[fname[:-4]].remove_node('')
    if i != 0:
        dtgraphs[fname[:-4]] = nx.compose(dtgraphs[fname[:-4]], dtgraphs[file_names[i][:-4]])
    i += 1

In [5]:
for gname in dtgraphs:
    if gname[-2:] == '-1':
        max_deg = max(dtgraphs[gname].degree(), key=lambda x: x[1])[1]
        if max_deg == 0:
            max_deg = 1
        posDG3D = nx.spring_layout(dtgraphs[gname], dim=3, k=1/np.sqrt(dtgraphs[gname].number_of_nodes())*10)
        partition = {node: 1 for node in dtgraphs[gname].nodes()}
        cmap = cm.get_cmap('Reds', max(partition.values()) + 1)
        export_data(dtgraphs[gname], posDG3D, partition, cmap, path='../nodes_edges/dynamic/', step=gname, max_deg=max_deg)
        G, d_hoverinfo, d_nodecolors, d_nodesize, posG3D = read_data(f'../nodes_edges/dynamic/nodes_{gname}.csv',
                                                                     f'../nodes_edges/dynamic/edges_{gname}.csv')
        node_trace = get_trace_nodes_3D_legend(posG3D, d_hoverinfo, d_nodecolors, d_nodesize)
        edge_trace = get_trace_edges_3D(G, posG3D, color = '#C7C7C7', opac = 0.4, linewidth=0.5)
        data = [*node_trace, edge_trace]
        plot_3D(data, path='../plots/dynamic/', fname = gname, scheme='dark')
    if gname == '2019-2':
        break

Opening in existing browser session.


[28968:28968:0100/000000.734084:ERROR:sandbox_linux.cc(377)] InitializeSandbox() called with multiple threads in process gpu-process.
[29037:29037:0420/165846.125911:ERROR:sandbox_linux.cc(377)] InitializeSandbox() called with multiple threads in process gpu-process.


Opening in existing browser session.


[29138:29138:0420/165847.845271:ERROR:sandbox_linux.cc(377)] InitializeSandbox() called with multiple threads in process gpu-process.


Opening in existing browser session.


...

[29277:29277:0420/165851.038571:ERROR:sandbox_linux.cc(377)] InitializeSandbox() called with multiple threads in process gpu-process.


Opening in existing browser session.


[29386:29386:0420/165853.590850:ERROR:sandbox_linux.cc(377)] InitializeSandbox() called with multiple threads in process gpu-process.


Opening in existing browser session.


KeyboardInterrupt: 